# QLearning Trader project overview

- [Overview Lecture](https://www.youtube.com/watch?v=K8xRATOpsqw)
- [Q & A](https://www.youtube.com/watch?v=f3bfmK8c3wA)

This is so that as a part of your strategy learner you can build Q learner

## Project Description

- till 12:29


## Thinking more: 12:30

- go through training data multiple times, with each pass improving your learner
- then test data will be used to call your API

- can look back to calculate your factors for the first day and that is ok
- Q Learner: DO NOTHING
  - you have some number of indicators and you should query your q learner and say, given the value for these indicators what should I do? Q Learner comes back and says "BUY"/"SELL"/"DO NOTHING"
  - Suppose it says "DO NOTHING"for next few days and we keep doing nothing
- Q Learner: BUY
  - Now suddenly one day it comes back with answer "BUY" and you should record this in the DF that you will return. Now things are different since you are holding the stock
  - now that you are in the market, you can make money or lose money and therefore you have keep track of if you are making money or losing money and then weave this into the `reward function` (which we will get in a moment)
- Q Learner: SELL (16:03)
  - Suppose we decided to exit after buy. Between buy and exit we accumulated some returns; 
  - Suppose we also shorted after we exited which we will eventually close by buying
  - for rest of the time period we are doing nothing! Here is what all that looks like in picture
  - <img alt="q_learning_1.png" src="images/q_learning_1.png" width="200"> <br/>

## How do we frame above discussion as RL problem? 17:09

- RL solves MDPs; what are the 4 things that you need to define MDPs?
  - 1. States
  - 2. Actions
  - 3. Rewards
  - 4. Transitions (which you may not know)

- Lets talk about each of those in the context of our problem:

## What is our state: 19:45
  - Our holdings/ Indicators => these are our states
- These are the things you need to know to decide if you want to buy or sell
- This is the state of the world you are considering as you decide whether you  want to buy or sell an asset

## Actions: 21:05
- BUY/SELL/DO NOTHING => these are my actions
- Instead of this you can also define your actions to be: LONG/NOTHING/SHORT
- Suppose you query your system and then it says, long but you are already holding shares and therefore do nothing
- Similarly say you are holding long and the system comes out with answer SHORT; then you would execute trade to close your position + short sell
- `Tip:` Remember the fewer actions you have and fewer states you have, all else being equal, your learner is going to converge quickly and perform better. 

## Don't go off of the world!

In robot navigation problem, when the learner might give position that may cause robot to move off the map; however, the code that takes the learners decision and moves it ensures that things likes these won't happen.

Similarly, trading Q learner tells you what to do but you need to consider what state you are in and if the given action should be executed. StrategyLearner should handle this despite what Q learner says. (26:40)

## Rewards: 27:37

What should the reward be?
- returns
- One of the reason RL is so well adapted to trading is because it is trying to find policy for how it is going to act and the reward directly matches our problem - we are trying to get good return!
- Now the question is when we entered a position and finally sold, when do you get that reward? (28:40) 
  - one view: wait till exit and get all reward at exit. This is just like our robot navigator where reward was finally granted when it reached the goal
  - second approach: I bought long; market went up today and therefore lets give it reward for single daily return. You are still holding and market goes slightly down and therefore you give a negative reward. In each day you get reward for what happened in previous day.
- Which will converge faster?  (29:45)
  - based on 2 rewards scheme above which will converge faster to a policy?
  - Daily returns!
  - reason is because you are giving reward each day, a feedback mechanism, to learn more quickly

- Question on 32 mins: Should the reward be percentage change or cash returns?
  - if you use cash return then for 100K portfolio vs 10K it will be different reward answer; so use percentage change of daily returns. This way you an use same policy into any portfolio size.

## Transitions:

- Q learning is model free learning and we are not doing that

## More details for each step: 31:29

### Representing the state: 33:44
- Suppose you picked 3 indicators that you think will be useful for buying and selling.
- Q learner use the Q table and it is sort of fixed dimension with respect to state and actions. But indicators are continuous and therefore you `need to discretize` these continuous values into integers.
  - for eg lets say for each indicator we are going to break each indicator into 10 possible discrete values. eg: my 3 indicators X1, X2 & X3
    - X1 -> discrete value between 0-9, say 2
    - X2 -> discrete value between 0-9, say 3
    - X3 -> discrete value between 0-9, say 5
    - see lecture on how to discretize
    - `Remember` there are certain params used to discretize and you will need to remember those params when you get to testing phase
  
## Stepping through the process 38:09

<img alt="q_learning_2.png" src="images/q_learning_2.png" width="200"> <br/>

### Strategy learner - add evidence part:
  - while not converged   #FOR_LOOP_A
    - X = calculate indicators for first day
    - querysetstate(X)  => (this completes very first day setup)
    - For each day (iteration)  (40:46)    #FOR_LOOP_B
      - <todo: 
        - calculate indicators and discretize them; 
        - calculate value of reward
        - query our learner>
      - r = calculate reward depending on what you are holding; this is 0 on first day
      - a = query q learner(X, r)   => give in new state and reward; ans is lONG/SHORT/NOTHING
      - [HERE YOU NEED TO IMPLEMENT THE SAID ACTION FOLLOWING THE RULE 'DON'T GO OFF']
        - ALSO add this action to your data frame of trades
      - x = new state (mentioned at 52:12)
    - With this go through entire training period (i.e #FOR_LOOP_B corresponds to one loop of the training data set)
    - OUT SIDE OF FOR_LOOP_B (but inside the #FOR_LOOP_A) check if converged else you are back to #FOR_LOOP_A & you go through the training data set again!

#### What does it mean to converge? 44:15
- 1. Policy isn't changing any longer; say for last 2 passes policy is same
  - can track actions for 2 pass and compare them to say if it has converged
  - or track returns and if it hasn't changed, then it probably converged!
- 2. Another way is to take snapshot of dataframe of trades before and after iteration; if it same, probably converged
- `Note:` Ensure you iterate at least some minimum number of times so that you are not randomly lucky! Also, have maximum number of iteration so that you do not run out of time

### Strategy learner - test policy part: 50:44

<img alt="q_learning_3.png" src="images/q_learning_3.png" width="200"> <br/>

Almost like add evidence part but few things taken off

- From add evidence part, take off the while not converged part since we don't care about it here
- From for each day part, take off calculation for rewards, actions, implementing actions;
  - instead call querysetstate(X) over and over again
  - THEN IMPLEMENT THE ACTION
  - add to df
  - X = new state
- with this test policy, you went through test data to generate orders which is what should be returned
- test case will check for the performance of portfolio based on this return value




53:58